In [1]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

from collections import defaultdict

In [75]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)
    
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']
participantFrames = [frame['participantFrames'] for frame in framesData]

# Access the 'frames' array within 'metadata'
framesData = data['info']['frames']

# Create dictionaries to store the counts and most mentioned names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []
participantChampionMapping = {}


In [32]:
targetPuuid = '92u7ZxgOLcJuu0VAAcp86ucO0c8ITXLkCilyh7PvlLr3D-D_9MGkWRVNLkzdgU-_GRMQLV7WstNILQ'

In [39]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]

    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)
#mostMentionedNamesDict


In [58]:
def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

participantData[1]


[2,
 '92u7ZxgOLcJuu0VAAcp86ucO0c8ITXLkCilyh7PvlLr3D-D_9MGkWRVNLkzdgU-_GRMQLV7WstNILQ',
 'MasterYi']

In [44]:
def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

# Example usage:
# Assuming 'targetPuuid' is the Puuid you want to find
targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")

Participant ID: 2, Champion: MasterYi


In [62]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData

def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key}:")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

In [90]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = (f"{targetParticipantId}")
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120031
championStats:
    armor: 36
    attackDamage: 71
    attackSpeed: 111
    ccReduction: 5
    health: 242
    healthMax: 763
    healthRegen: 23
    magicResist: 33
    movementSpeed: 355
    power: 261
    powerMax: 281
    powerRegen: 23
  currentGold: 161
damageStats:
    totalDamageDone: 3145
    totalDamageTaken: 949
  jungleMinionsKilled: 4
  level: 2
  participantId: 2
position:
    x: 7927
    y: 2312
  totalGold: 611
  xp: 325

---

